<a href="https://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/making_speech_dataset_with_Auditok_es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cuaderno Colab para fragmentar audio con[auditok](https://github.com/amsehili/auditok).
## Muy útil para hacer conjuntos de datos de voz de forma automática

Cuaderno hecho por [rmcpantoja](https://github.com/rmcpantoja/)

## Primeros pasos

In [ ]:
#@title Montar tu Google Drive
#@markdown Esto será útil si quieres guardar tu conjunto de datos generado a tu [drive](http://drive.google.com/).
from google.colab import drive
drive.mount('drive', force_remount=True)

In [ ]:
#@title instalar software
#@markdown Aquí instalaremos y actualizaremos los paquetes de pip para Python, así como el repositorio de la herramienta, que es lo más importante.

#@markdown * Nota: por favor, reinicia el entorno de jecución cuando la celda termine de ejecutarse, caso contrario obtendremos errores durante el proceso.

!git clone https://github.com/amsehili/auditok.git
%cd auditok
!python setup.py install
%cd ..
!pip install --upgrade gdown
!pip install yt-dlp pydub

!mkdir project

## Comenzando a interactuar con la herramienta

In [ ]:
from google.colab import files
import yt_dlp

def download_from_youtube(yt_url):
  ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'wav',
          'preferredquality': '320',
      }],
  }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      ydl.download([yt_url])

#@title Obtener datos de audio de la voz deseada

#@markdown Actualmente, hay tres formas de obtener audio de voz:
#@markdown * Descargar desde YouTube: Con esta opción podrás descargar multimedia de YouTube y muchos sitios como Facebook y más.
#@markdown * Descargar mediante ID de Google Drive: Te permite descargar el audio exclusivamente desde Drive.
#@markdown * Subir: Aparecerá un marco donde te permitirá cargar el archivo de audio a colab.

#@markdown En el campo "input", tendrás que ingresar un enlace o ID de Drive. Al seleccionar la opción "subir", este campo no se tendrá en cuenta. Sin embargo, puedes omitir esta celda si vas a subir un audio en el panel izquierdo en colab.

#@markdown **Selecciona la forma en la que te gustaría obtener el audio:**
mode = "descargar mediante un ID de Drive" #@param ["descargar desde youtube (yt-dlp)", "descargar mediante un ID de Drive", "Subir"]
input = "Enlace o ID a descargar" #@param {type:"string"}
if mode == "descargar desde youtube (yt-dlp)" and "http" in input:
  download_from_youtube(input)
elif mode == "descargar mediante un ID de Drive" and input.startswith("1"):
  !gdown {input}
elif mode == "Subir":
  input = files.upload()
print("¡Hecho!")

In [ ]:
#@title Configuraciones
#@markdown Aquí puedes hacer tus configuraciones preferidas para la herramienta, como manipular el rango de la duración de los fragmentos de audio y la detección de silencio.
#@markdown ---

#@markdown **Selecciona la duración mínima**
min_duration = 10.0 #@param {type:"number"}
#@markdown ---
#@markdown **Selecciona la duración máxima**
max_duration = 15.5 #@param {type:"number"}
#@markdown ---
#@markdown **Duración máxima de silencios**
max_silence = 0.5 #@param {type:"number"}
#@markdown ---

#@markdown **Ajusta el humbral de detección**
threshold = 55 #@param {type:"integer"}
#@markdown ---

In [ ]:
#@title ¡Hora de comenzar el trabajo!
#@markdown Pero primero, aquí hay unas cosas para configurar la forma en que el cuaderno guardará el conjunto de datos:

#@markdown **Ubicación del archivo de audio previamente descargado o subido que contiene la voz**
audio_dir = "audio.wav" #@param {type:"string"}
#@markdown ---

#@markdown **¿Comprimir como un archivo zip?**
compress = True #@param {type:"boolean"}
#@markdown ---

#@markdown **Si se va a comprimir como un archivo zip, ¿Dónde guardarlo?**
export_dir = "/content/drive/MyDrive/voice1" #@param {type:"string"}
#@markdown ---
import auditok
print("Dividiendo audio... esto puede llevar un poco de tiempo.\n")
# split returns a generator of AudioRegion objects
audio_regions = auditok.split(
    audio_dir,
    min_dur=min_duration,     # minimum duration of a valid audio event in seconds
    max_dur=max_duration,       # maximum duration of an event
    max_silence=max_silence, # maximum duration of tolerated continuous silence within an event
    energy_threshold=threshold # threshold of detection
)

for i, r in enumerate(audio_regions):

    # Regions returned by `split` have 'start' and 'end' metadata fields
    #print("Region {i}: {r.meta.start:.3f}s -- {r.meta.end:.3f}s".format(i=i, r=r))

    # play detection
    # r.play(progress_bar=True)

    # region's metadata can also be used with the `save` method
    # (no need to explicitly specify region's object and `format` arguments)
    filename = r.save("project/region_{meta.start:.3f}-{meta.end:.3f}.wav")
    #print("region saved as: {}".format(filename))
import os
n_wavs = len(os.listdir("/content/project"))
print(f"¡Hecho! La división se ha realizado en {n_wavs} audios.")
if compress:
  print("Comprimiendo audios...\n")
  import os
  if not os.path.exists(export_dir):
    os.makedirs(export_dir)
  import shutil
  shutil.make_archive(export_dir+"/wavs", 'zip', '/content/project')
  print("¡Compresión finalizada!")
print("Todas las tareas finalizadas.")

# Cuadernos relacionados

¿Te gustaría transcribir estos audios? ¡Usa el cuaderno de transcripción de Whisper!

* [Cuaderno en Español](https://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/OpenAI%20Whisper%20-%20DotCSV%20(Speech%20dataset%20multi-transcryption%20support)es.ipynb)
* [Cuaderno en Francés](https://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/OpenAI%20Whisper%20-%20DotCSV%20(Speech%20dataset%20multi-transcryption%20support)fr.ipynb)
* [Cuaderno en Inglés](https://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/OpenAI%20Whisper%20-%20DotCSV%20(Speech%20dataset%20multi-transcryption%20support)en.ipynb)